In [1]:
from datasets import load_dataset
import torch
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import mean_squared_error
from datasets import load_dataset
from transformers import BertTokenizer, BertForQuestionAnswering
from transformers import BertTokenizerFast, TFBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

In [2]:
dataset = load_dataset("commonsense_qa")

Using the latest cached version of the module from C:\Users\Mia\.cache\huggingface\modules\datasets_modules\datasets\commonsense_qa\28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b (last modified on Mon Dec 25 12:04:23 2023) since it couldn't be found locally at commonsense_qa., or remotely on the Hugging Face Hub.
Found cached dataset commonsense_qa (C:/Users/Mia/.cache/huggingface/datasets/commonsense_qa/default/1.0.0/28d68f56649a7f0c23bc68eae850af914aa03f95f810011ae8cf58cc5ff5051b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
generics_kb = load_dataset("generics_kb", "generics_kb")

Using the latest cached version of the module from C:\Users\Mia\.cache\huggingface\modules\datasets_modules\datasets\generics_kb\9b41cde494db24f842a9260588bcfb2e3a257364568666ef240e98c70fb0e709 (last modified on Wed Jan 10 14:22:24 2024) since it couldn't be found locally at generics_kb., or remotely on the Hugging Face Hub.
Found cached dataset generics_kb (C:/Users/Mia/.cache/huggingface/datasets/generics_kb/generics_kb/1.0.0/9b41cde494db24f842a9260588bcfb2e3a257364568666ef240e98c70fb0e709)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
train_ds = dataset['train'][:5000]
test_ds = dataset['test']
val_ds = dataset['validation']

In [5]:
questions = train_ds['question']
choices = [choice['text'] for choice in train_ds['choices']]
answers = train_ds['answerKey']

In [6]:
kb_sentences = generics_kb['train']['generic_sentence']

In [7]:
# decrease the size of the knowledge base to speed up encoding
kb_sentences = random.sample(kb_sentences, 1000)

In [8]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [9]:
len(kb_sentences)

1000

In [10]:
max_len = 27

In [11]:
knowledge_base_tokens = [tokenizer.encode(sentence, return_tensors='pt')[0] for sentence in kb_sentences]

In [12]:
knowledge_base_tokens_padded = torch.stack([
    torch.nn.functional.pad(token, (0, max_len - len(token)))
    for token in knowledge_base_tokens
])

In [13]:
def convert_answer_for_index(i, ds):
    answer_index = ord(ds['answerKey'][i]) - ord('A')
    return ds['choices'][i]['text'][answer_index] 

In [14]:
sentences_merged = []

for i in range(len(questions)):
    question = questions[i]
    choices_i = choices[i]

    question_tokens = tokenizer.encode(question, return_tensors='pt')[0]
    question_tokens = torch.nn.functional.pad(question_tokens, (0, max_len - len(question_tokens)))

    # print("Question Tokens Shape:", question_tokens.shape)
    # print("Knowledge Base Tokens Shape:", knowledge_base_tokens_padded[0].shape)

    similarities = [cosine_similarity(question_tokens.detach().reshape(1, -1).numpy(), kb_token.reshape(1, -1).detach().numpy())[0][0] for kb_token in knowledge_base_tokens_padded]

    most_similar_index = np.argmax(similarities)
    selected_kb_sentence = kb_sentences[most_similar_index]

    input_text = f"{question} {selected_kb_sentence}"
    sentences_merged.append(input_text)

In [15]:
len(sentences_merged)

5000

In [16]:
sequences = []
answers_converted = [convert_answer_for_index(i, train_ds) for i in range(0, len(answers))]
for sentence, answer in zip(sentences_merged, answers_converted):
        sequences.append(f'{sentence} - {answer}')

In [17]:
len(sequences)

5000

In [18]:
tokenized_data = tokenizer(
    sequences,
    padding='max_length',  
    return_tensors='tf',
    truncation=True,
    max_length=100 
)

In [19]:
len(tokenized_data['input_ids'])

5000

In [20]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [21]:
input_ids = Input(shape=(100,), name='input_token', dtype='int32')
att_masks = Input(shape=(100,), name='masked_token', dtype='int32')
bert_in = bert_model(input_ids, attention_mask=att_masks)[1]
answer_output = Dense(5, activation='softmax', name='answer')(bert_in)

In [22]:
model = Model(inputs=[input_ids, att_masks], outputs=[answer_output])
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
print(len(tokenized_data['input_ids']))
print(len(tokenized_data['attention_mask']))
print(len(answers))

5000
5000
5000


In [24]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [25]:
answer_data = np.array(answers).reshape(-1, 1)
encoder = OneHotEncoder(categories=[['A', 'B', 'C', 'D', 'E']])
encoded_answer_data = encoder.fit_transform(answer_data).toarray()

In [26]:
model.fit([tokenized_data['input_ids'], tokenized_data['attention_mask']], encoded_answer_data, epochs=3, batch_size=32)

Epoch 1/3


157/157 [==============================] - 2662s 17s/step - loss: 2.3211 - accuracy: 0.2062
Epoch 2/3
157/157 [==============================] - 4440s 28s/step - loss: 2.0864 - accuracy: 0.1906
Epoch 3/3
157/157 [==============================] - 2740s 17s/step - loss: 1.9461 - accuracy: 0.1964


In [27]:
pred_tokens = tokenizer(val_ds['question'], max_length=100, return_tensors='np', truncation=True, pad_to_max_length=True)

c:\Users\Mia\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [28]:
pred_tokens['attention_mask'].shape

(1221, 100)

In [29]:
pred = model.predict([pred_tokens['input_ids'], pred_tokens['attention_mask']])

39/39 [==============================] - 215s 5s/step


In [30]:
y_pred = []
answerKeys = ['A', 'B', 'C', 'D', 'E']

for p in pred:
    y_pred.append(answerKeys[np.argmax(p)])

In [31]:
y_test = val_ds['answerKey']

In [32]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluation Metrics

In [42]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=1)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.19246519246519248
Precision: 0.8384930384930385
Recall: 0.2
F1 Score: 0.06456043956043957
